In [ ]:
%%time
%%bash --out output --err error
source /home/olivier/Devel/venvs/bonndit/bin/activate
shore-deconvolve.py /home/olivier/Devel/test/samuel/in4michi/data.nii.gz \
                    /home/olivier/Devel/test/samuel/in4michi/bonndit_result/response.npz \
                    /home/olivier/Devel/test/samuel/in4michi/michi_result/odfnoconstraint.nrrd \
                    --pos none \
                    --csf /home/olivier/Devel/test/samuel/in4michi/michi_result/csfnoconstraint.nrrd \
                    --gm /home/olivier/Devel/test/samuel/in4michi/michi_result/gmnoconstraint.nrrd \
                    --wm /home/olivier/Devel/test/samuel/in4michi/michi_result/wmnoconstraint.nrrd \

In [14]:
import subprocess

In [55]:
SCRIPT = 'shore-deconvolve.py '
DATA_DIR = '/home/olivier/Devel/test/samuel/in4michi'
OUT_DIR ='/home/olivier/Devel/test/samuel/in4michi/test_results'
OPTIONS = ['hpsd', 'nonneg', 'none']
venv = '. /home/olivier/Devel/venvs/bonndit/bin/activate && '

for option in OPTIONS:
    command = venv + SCRIPT + '{di}/data.nii.gz {out}/response.npz \
{out}/odf{opt}.nrrd --pos {opt} --csf {out}/csf{opt}.nrrd \
--gm {out}/gm{opt}.nrrd --wm {out}/wm{opt}.nrrd'.format(di=DATA_DIR, opt=option, out=OUT_DIR)
    out = check_output(command, shell = True)
    #print(command)

out

b'order (radial / angular): 4 / 4\n[[281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n [281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n [281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n ...\n [ 93.2258914    0.           0.         ...   0.          82.49415443\n   29.13357179]\n [ 93.2258914    0.           0.         ...   0.          82.49415443\n   29.13357179]\n [ 93.2258914    0.           0.         ...   0.          82.49415443\n   29.13357179]]\nCondition number of M: inf\nOptimizing...\n  0.0%\x08\x08\x08\x08\x08\x08\x08\x08   done   \n'

In [36]:
p = subprocess.Popen('python --version', shell=True, stdout=subprocess.PIPE)

In [37]:
out, err = p.communicate()
out

b''

In [39]:
venv.split() + 'python --version'.split()

['.',
 '/home/olivier/Devel/venvs/bonndit/bin/activate',
 '&&',
 'python',
 '--version']

In [51]:
from subprocess import check_output
errout = c
errout

b'Python 3.5.2\n'